<a href="https://colab.research.google.com/github/Inteli-College/2024-2A-T01-CC11-G04/blob/62-atualizao-da-pipeline-de-processamento/notebooks/sprint3/pre_processing_remake_analisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
!pip install jedi
!pip install -U spacy -q
!pip install -U pip setuptools wheel -q
!python -m spacy download pt_core_news_sm -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.0 which is incompatible.
pt-core-news-sm 3.7.0 requires spacy<3.8.0,>=3.7.0, but you have spacy 3.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 78.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [41]:
import os
import re
import nltk
import spacy
import string
import pandas as pd
import matplotlib.pyplot as plt

from copy import Error
from enum import Enum, auto
from fuzzywuzzy import fuzz
from datetime import datetime
from spacy.tokens import Token
from collections import Counter
from google.colab import runtime
from nltk.stem import RSLPStemmer
from nltk.corpus import stopwords
from abc import ABC, abstractmethod
from typing import List, Type, Union
from spacy.tokenizer import Tokenizer
from spacy.lang.xx import MultiLanguage
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from multiprocessing import Pool, cpu_count

In [43]:
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
# Testando importação de arquivos necessários

# Stop words
with open('/content/drive/MyDrive/modulo11/ptbr-dict/personalized_stop_words_ptbr.txt', 'r', encoding='utf-8') as file:
    stop_words = set([line.strip() for line in file.readlines()])
print(stop_words)

# Dicionário pt-br
with open("/content/drive/MyDrive/modulo11/ptbr-dict/pt-br_dict", 'r') as dict_file:
    aurelio = [palavra.lower() for palavra in dict_file.read().split("\n")]
print(aurelio[:50])

# Palavras relevantes por classe
with open("/content/drive/MyDrive/modulo11/ptbr-dict/main_words.txt", 'r') as main_words_file:
    relevant_words = set([palavra.lower() for palavra in main_words_file.read().split("\n")])
print(relevant_words)

{'esse', 'onde', 'para', 'nossas', 'quem', 'te', 'aquilo', 'vosso', 'em', 'nos', 'as', 'essa', 'lhes', 'ele', 'do', 'minhas', 'aquela', 'esta', 'algo', 'ti', 'seus', 'este', 'consigo', 'me', 'eu', 'estes', 'a', 'suas', 'teus', 'aqueles', 'ela', 'seu', 'vossas', 'quais', 'aquele', 'de', 'vos', 'o', 'um', 'tuas', 'estas', 'qual', 'lhe', 'tu', 'essas', 'meus', 'alguém', 'tua', 'elas', 'da', 'pra', 'no', 'ainda', 'isto', 'nossa', 'por', 'meu', 'se', 'na', 'quanto', 'contigo', 'minha', 'mim', 'vossa', 'comigo', 'isso', 'teu', 'esses', 'si', 'vossos', 'que', 'vós', 'nosso', 'nossos', 'aquelas', 'os', 'como', 'nós', 'sua', 'eles'}
['a', 'ª', 'à', 'á', 'ã', 'a-amilase', 'á-bê-cê', 'á-é-i-ó-u', 'a-histórico', 'à-propos', 'à-toa', 'a.a.', 'a.c.', 'a.c.', 'a.m.', 'aa.', 'aabora', 'aac', 'aaiún', 'aaleniano', 'aaleniense', 'aalênio', 'aalense', 'aaquenense', 'aaquênio', 'aarão', 'aardvark', 'aariano', 'aarônico', 'aarônida', 'aaronita', 'ab', "ab'sáber", 'ab-henry', 'ab-reação', 'ab-reagir', 'ab-r

# Pré-Processamento

## Pipeline crua

In [90]:
class NLPLibrary(Enum):
    """
    Enum for supported NLP libraries.
    """
    nltk = 0
    spacy = 1

class TextProcessingStep(ABC):
    """
    Base class for a text processing step.
    """

    @abstractmethod
    def execute(self, data: Union[str, List[str]]) -> Union[str, List[str]]:
        """
        Executes the processing step.
        """
        pass

    @abstractmethod
    def requires(self) -> List[Type['TextProcessingStep']]:
        """
        Specifies steps that must be executed before this one.
        """
        pass

class PreprocessingTextPipeline:
    """
    Pipeline for managing and executing text preprocessing steps.
    """

    def __init__(self, nlpLibrary: NLPLibrary):
        """
        Initializes the pipeline.
        """
        self.steps = []
        self.nlpLibrary = nlpLibrary

    def add_step(self, step: TextProcessingStep):
        """
        Adds a step to the pipeline, checking dependencies.
        """
        for required_step in step.requires():
            if not any(isinstance(s, required_step) for s in self.steps):
                raise ValueError(f"{step.__class__.__name__} requires {required_step.__name__} to be added first.")

        self.steps.append(step)

    def run(self, data: Union[str, List[str]]) -> Union[str, List[str]]:
        """
        Executes all steps in the pipeline.
        """
        for step in self.steps:
            data = step.execute(data, self.nlpLibrary)
        return data


In [91]:
class RemoveEscapeSequences(TextProcessingStep):
    """
    Removes escape sequences from the text.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        return data.translate(str.maketrans({'\n': ' ', '\t': ' ', '\r': ' '}))

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class Lowercase(TextProcessingStep):
    """
    Converts all text to lowercase.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        return data.lower()

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class RemoveNumbers(TextProcessingStep):
    """
    Removes numbers from the text.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        return re.sub(r'\d+', '', data)

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class RemovePunctuation(TextProcessingStep):
    """
    Removes punctuation from the text.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        translator = str.maketrans('', '', string.punctuation)
        return data.translate(translator)

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class Tokenization(TextProcessingStep):
    """
    Tokenizes the text into words.
    """
    def execute(self, data: str, library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            return word_tokenize(data)
        elif library == NLPLibrary.spacy:
            tokens = MultiLanguage().tokenizer(data)
            return [token.text for token in tokens]

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class RemoveStopwords(TextProcessingStep):
    """
    Removes stopwords from the text.
    """
    def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
        with open('/content/drive/MyDrive/modulo11/ptbr-dict/personalized_stop_words_ptbr.txt', 'r', encoding='utf-8') as file:
            stop_words = set([line.strip() for line in file.readlines()])
        return [word for word in data if word.lower() not in stop_words]

    def requires(self) -> List[Type[TextProcessingStep]]:
        return [Tokenization]

class Stemming(TextProcessingStep):
    """
    Applies stemming to the text.
    """
    def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            stemmer = RSLPStemmer()
            return [stemmer.stem(word) for word in data]
        elif library == NLPLibrary.spacy:
            raise Error("spacy does not provide built-in stemming resources")

    def requires(self) -> List[Type[TextProcessingStep]]:
        return [Tokenization, RemoveStopwords]

class Lemmatization(TextProcessingStep):
    """
    Applies lemmatization to the text.
    """
    def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            raise Error("nltk does not provide built-in pt-br lemmatizer resources")
        elif library == NLPLibrary.spacy:
            nlp = spacy.load("pt_core_news_sm")
            doc = nlp(" ".join(data))
            return [token.lemma_ for token in doc]

    def requires(self) -> List[Type[TextProcessingStep]]:
        return [Tokenization, RemoveStopwords]

class OrtographicCorrection(TextProcessingStep):
  """
  Performs orthographic correction to a few words in the doc
  """
  # abrir o aurelio
  # comparar se a palavra está contida no dict
  def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
      # dicionário pt-br
      with open("/content/drive/MyDrive/modulo11/ptbr-dict/pt-br_dict", 'r') as dict_file:
          aurelio = [palavra.lower() for palavra in dict_file.read().split("\n")]
      # palavras principais por classe
      with open("/content/drive/MyDrive/modulo11/ptbr-dict/main_words.txt", 'r') as main_words_file:
          relevant_words = set([palavra.lower() for palavra in main_words_file.read().split("\n")])
      for i, word in enumerate(data):
          if word not in aurelio_set:
              # Compara a palavra com uma das principais
              for relevant_word in relevant_words:
                  similarity = fuzz.ratio(word, relevant_word)
                  if similarity > 70 and word != relevant_word:
                      data[i] = relevant_word
                      print(f"Substituted \"{word}\" to \"{relevant_word}\"")
                      print(similarity)
      return(data)
  def requires(self) -> List[Type[TextProcessingStep]]:
      return [Tokenization, RemoveStopwords]

# Testes e melhorias

In [12]:
# Build the sequence that belongs to Pipeline
pipeline = PreprocessingTextPipeline(NLPLibrary.spacy)

pipeline.add_step(Lowercase())
pipeline.add_step(Tokenization())
pipeline.add_step(RemoveStopwords())

## Verificando tokens fora do dicionário Aurélio

In [13]:
strTest = "Quero enviar uma remessa de dinheiro para o Japão"
tokensTest = strTest.split(" ")
tokensTest

['Quero', 'enviar', 'uma', 'remessa', 'de', 'dinheiro', 'para', 'o', 'Japão']

In [14]:
aurelio: str = ""
with open("/content/drive/MyDrive/modulo11/ptbr-dict/pt-br_dict", 'r') as arquivo:
  aurelio = arquivo.read()

aurelio[:50]

'a\nª\nà\ná\nã\na-amilase\ná-bê-cê\ná-é-i-ó-u\na-histórico\n'

In [15]:
type(aurelio)

str

In [16]:
aurelio_list = aurelio.split("\n")
aurelio_set = set([palavra.lower() for palavra in aurelio_list])

tokensValidos = []
tokensInvalidos = []

for token in tokensTest:
  if token.lower() in aurelio_set:
    tokensValidos.append(token)
  else:
    tokensInvalidos.append(token)

In [17]:
print(tokensValidos)
print(tokensInvalidos)

['Quero', 'enviar', 'uma', 'de', 'dinheiro', 'para', 'o', 'Japão']
['remessa']


## Extraindo as principais palavras de cada classe

Extraindo as palavras únicas de cada classificação de intent (utilizando dados augmentados)

In [84]:
aug_data = pd.read_csv('/content/drive/MyDrive/modulo11/resources/data_2024-09-12_16-38-24.csv')
aug_data.sample(3)

,intent,question,processed_question
1464,Como fazer remessa,"Saldo no app ok, e agora, como faço pra enviar...","saldo,app,ok,agora,faço,enviar,pro,brasil"
1102,Problemas de remessa,"Mandei a grana já faz um tempinho, mas até ago...","mandei,grana,já,faz,tempinho,mas,até,agora,nad..."
5036,Tempo de entrega do cartao,Boa tarde. Realizei um pedido do cartão de rem...,"boa,tarde,realizei,pedido,cartão,remessa,brast..."


In [85]:
len(aug_data['intent'].unique())

17

In [86]:
def count_words_per_intent(df, column_intent='intent', column_question='processed_question', top_n=10):
  intents = df[column_intent].unique()
  palavra_intent = {}

  for intent in intents:
    intents = df[column_intent].unique()
    palavra_intent = {}

    for intent in intents:
      # Filtrar as questões de uma determinada intenção
      questoes_intent = df[df[column_intent] == intent][column_question]

      # Transformar a lista de tokens em uma lista de palavras
      todas_palavras = []
      for questoes in questoes_intent:
          todas_palavras.extend(questoes.split(','))  # Supondo que as palavras estão separadas por vírgula

      # Contar a frequência das palavras
      contador = Counter(todas_palavras)

      # Obter as palavras mais comuns
      palavra_intent[intent] = contador.most_common(top_n)

    return palavra_intent

In [87]:
palavras_por_intent = count_words_per_intent(aug_data)
palavras_por_intent

{'Acesso a conta': [('senha', 204),
  ('não', 163),
  ('conta', 140),
  ('acessar', 98),
  ('estou', 74),
  ('brastel', 57),
  ('com', 54),
  ('mas', 47),
  ('preciso', 46),
  ('app', 45)],
 'Atualizacao de dados cadastrais': [('endereço', 181),
  ('atualizar', 86),
  ('cadastro', 64),
  ('app', 63),
  ('dados', 63),
  ('aplicativo', 57),
  ('faço', 51),
  ('preciso', 45),
  ('favor', 44),
  ('gostaria', 38)],
 'Cadastro de beneficiario': [('beneficiário', 188),
  ('conta', 128),
  ('\n', 119),
  ('brasil', 74),
  ('preciso', 68),
  ('dados', 68),
  ('é', 56),
  ('enviar', 55),
  ('cadastro', 53),
  ('cartão', 51)],
 'Problemas de remessa': [('não', 250),
  ('conta', 149),
  (' ', 129),
  ('dia', 111),
  ('remessa', 105),
  ('até', 72),
  ('brasil', 71),
  ('mas', 69),
  ('agora', 65),
  ('fiz', 59)],
 'Termos e condicoes do servico': [(' ', 95),
  ('é', 90),
  ('brasil', 90),
  ('dinheiro', 82),
  ('brastel', 77),
  ('saber', 67),
  ('conta', 66),
  ('enviar', 63),
  ('pix', 54),
  ('

In [89]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Criação do TF-IDF vectorizer
tfidf = TfidfVectorizer()

# Fit o TF-IDF nas perguntas processadas
tfidf_matrix = tfidf.fit_transform(aug_data['processed_question'])

# Obtenha as palavras (tokens) e o índice de classes
words = tfidf.get_feature_names_out()
intentions = aug_data['intent']

# Converter a matriz TF-IDF para um DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=words)

# Agora, adicionar a coluna de classes (intentions) ao DataFrame
tfidf_df['intent'] = intentions

# Função para extrair as top 5 palavras mais relevantes para todas as classes
def get_top_words_all_classes(tfidf_df, top_n=5):
    # Obter todas as classes únicas
    unique_classes = tfidf_df['intent'].unique()

    # Dicionário para armazenar as top words por classe
    top_words_by_class = {}

    # Iterar sobre cada classe
    for class_label in unique_classes:
        class_data = tfidf_df[tfidf_df['intent'] == class_label].drop(columns=['intent'])
        # Média dos TF-IDF scores por palavra para a classe específica
        mean_tfidf = class_data.mean(axis=0)
        # Selecionar as top N palavras mais relevantes
        top_words = mean_tfidf.sort_values(ascending=False).head(top_n)
        # Adicionar ao dicionário
        top_words_by_class[class_label] = top_words

    return top_words_by_class

# Chamando a função para extrair as 5 top words para todas as classes
top_words_all_classes = get_top_words_all_classes(tfidf_df, top_n=5)

# Exibir as top words para cada classe
for classe, top_words in top_words_all_classes.items():
    print(f"Palavras mais relevantes para a classe {classe}:")
    print(top_words)
    print("\n")


Palavras mais relevantes para a classe Acesso a conta:
senha      0.176734
acessar    0.103923
não        0.092898
conta      0.086739
estou      0.055247
dtype: float64


Palavras mais relevantes para a classe Atualizacao de dados cadastrais:
endereço     0.212026
atualizar    0.117164
cadastro     0.081951
app          0.072494
dados        0.066841
dtype: float64


Palavras mais relevantes para a classe Cadastro de beneficiario:
beneficiário    0.150520
conta           0.054434
cadastro        0.047132
dados           0.046380
cadastrar       0.044459
dtype: float64


Palavras mais relevantes para a classe Problemas de remessa:
não        0.128734
conta      0.071605
dia        0.067801
até        0.060508
remessa    0.056793
dtype: float64


Palavras mais relevantes para a classe Termos e condicoes do servico:
brasil      0.054969
dinheiro    0.052786
brastel     0.052033
pix         0.051017
saber       0.044762
dtype: float64


Palavras mais relevantes para a classe Como fazer re

Podemos ver algumas stop words a serem removidas também

# Lehvestein distance

Tolerância a erros de digitação: Levenshtein considera pequenas alterações em uma palavra, como a troca de letras ou a falta de uma letra, o que é típico de erros de digitação.

Simplicidade: Não requer treinamento ou embeddings, o que o torna rápido e eficiente quando você apenas precisa verificar palavras em uma lista.

Boa precisão para pequenos erros: Funciona bem para erros pequenos, como no seu exemplo "remeça" e "remessa", onde apenas uma letra foi modificada.

In [29]:
!pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.7 MB/s eta 0:00:00


In [36]:
from fuzzywuzzy import fuzz

# Exemplo de lista de palavras importantes
palavras_importantes = ["remessa", "envio", "transporte", "entrega"]

# Palavra digitada erroneamente
palavra_checada = "remeça"

# Função para checar a similaridade entre a palavra checada e as palavras importantes
def checar_similaridade(palavra_checada, palavras_importantes):
    similaridades = {}
    for palavra in palavras_importantes:
        similaridade = fuzz.ratio(palavra_checada, palavra)  # Similaridade baseada na distância de Levenshtein
        similaridades[palavra] = similaridade
    return similaridades

In [38]:
palavra_checada = "remeça"
palavras_importantes = ["remessa", "envio", "senha"]
# Checando a similaridade
resultados_similaridade = checar_similaridade(palavra_checada, palavras_importantes)

# Exibir as palavras mais semelhantes
print("Similaridade com as palavras importantes:")
for palavra, similaridade in resultados_similaridade.items():
    print(f"{palavra}: {similaridade}%")

Similaridade com as palavras importantes:
remessa: 77%
envio: 18%
senha: 36%


# Juntando tudo

Depois do TO-IDF já feito para o conjunto de dados:

1. Remoção das stop words personalizadas
2. Chegagem das palavras que não estão contidas no arquivo `pt-br_dict.txt` (dicionário da língua portuguesa)
3. Se uma palavra não está contida no arquivo dicionário, sua similaridade com palavras relevantes (`main_words.txt`) será checada
4. Se a similaridade for calculada como alta, ela será substituída pela palvra relevante em questão

In [35]:
test_sentence = "enviei dinheiro mas ainda não caiu na minha cnota"

In [64]:
# Build the sequence that belongs to Pipeline
pipeline = PreprocessingTextPipeline(NLPLibrary.spacy)

pipeline.add_step(Lowercase())
pipeline.add_step(RemoveNumbers())
pipeline.add_step(RemovePunctuation())
pipeline.add_step(Tokenization())
pipeline.add_step(RemoveStopwords())
pipeline.add_step(OrtographicCorrection())

In [56]:
pipeline.run(data=test_sentence)

Substituted "cnota" to "conta"
80


['enviei', 'dinheiro', 'mas', 'não', 'caiu', 'conta']

# Rodando o pré-processamento para a base de dados inteira e extraindo novamente as palavras principais

Isso será executado pois existiam muitas stop_words ainda nas palavras mais relevantes por classe. Após a identificação das palavras relevantes sem stop_words, serão separadas as palavras principais novamente, a lista de `relevant_words` será atualizada e o pré-processamento irá rodar novamente.

In [92]:
# Build the sequence that belongs to Pipeline
pipeline = PreprocessingTextPipeline(NLPLibrary.spacy)

pipeline.add_step(Lowercase())
pipeline.add_step(RemoveNumbers())
pipeline.add_step(RemovePunctuation())
pipeline.add_step(Tokenization())
pipeline.add_step(RemoveStopwords())
pipeline.add_step(OrtographicCorrection())

In [93]:
df = pd.read_excel("/content/drive/MyDrive/modulo11/resources/data_augmented.xlsx")
df.sample(3)

,intention,question,processed_question
2142,Como depositar,Preciso de orientação sobre o registro do my n...,"tem,que,fazer,alguma,coisa,com,o,my,number,pra..."
674,Cadastro de beneficiario,"Oi, tudo certo? Preciso excluir um contato, vo...","oi,tudo,certo,preciso,excluir,um,contato,você,..."
2378,Confirmacao de cambio/taxas,Será que você consegue ver aí quanto dá 400 re...,"por,favor,você,poderia,informar,quanto,é,o,tot..."


In [80]:
len(df)

5100

In [94]:
%%time

def process_question(question):
    # Process a question using the NLP pipeline and join the results with commas.
    return ','.join(pipeline.run(question))

df.columns = ['intent', 'question', 'processed_question']  # Rename the DataFrame columns.

df = df.drop(columns='processed_question')

with Pool(cpu_count()) as pool:
    # Process each question in parallel using the CPU cores.
    df['processed_question'] = pool.map(process_question, df['question'])

processed_df = df[['intent', 'question', 'processed_question']]  # Select relevant columns.

A saída de streaming foi truncada nas últimas 5000 linhas.
Substituted "dando" to "dados"
80
Substituted "remessa" to "remessas"
93
Substituted "atualizei" to "atualizar"
78
Substituted "remessa" to "remessas"
93
Substituted "brastel" to "brasil"
77
Substituted "remessa" to "remessas"
93
Substituted "brastelnfala" to "brastel"
74
Substituted "certo" to "cartão"
73
Substituted "dianquero" to "dinheiro"
71
Substituted "ocorreu" to "correio"
71
Substituted "acesso" to "acessar"
77
Substituted "senhannão" to "senha"
71
Substituted "remessa" to "remessas"
93
Substituted "remessa" to "remessas"
93
Substituted "remessa" to "remessas"
93
Substituted "remessas" to "remessa"
93
Substituted "usando" to "saldo"
73
Substituted "remessa" to "remessas"
93
Substituted "dianquero" to "dinheiro"
71
Substituted "brastel" to "brasil"
77
Substituted "cartãonnão" to "cartão"
75
Substituted "depositei" to "depósito"
71
Substituted "depositei" to "deposito"
82
Substituted "depositei" to "depósito"
71
Substitu

In [95]:
processed_df

import pandas as pd
from datetime import datetime

# Gerar o nome do arquivo com a data e horário atual
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"/content/drive/MyDrive/modulo11/resources/data_{current_time}.csv"

# Salvar o DataFrame em um arquivo CSV
processed_df.to_csv(file_name, index=False)

file_name  # Retorna o nome do arquivo gerado

'/content/drive/MyDrive/modulo11/resources/data_2024-09-12_22-47-29.csv'

Observações relevantes:

Apesar de estarem implementados, os passos de stemming e lemmatização não estão sendo utilizados no pré-processamento. Porém, por conta da correção ortográfica, podemos observar que algumas variações das palavras relevantes estão sendo reduzidas a apenas uma (contida no arquivo `main_words.txt`).

Essa substituição ocorre apenas com as palavras relevantes, visto que é calculada a similiridade.

É importante ressaltar também que isso cria uma capacidade menor de generalização por parte da rede que será treinada com esses dados processados, pois não serão fornecidas muitas variações das palavras mais relevantes. Por conta disso, será necessária fazer a correção ortográfica antes de cada predição da rede já treinada.